In [6]:
# ========================================
# 1) Install dependencies
# ========================================
!pip install -q efficientnet tensorflow scikit-learn

# ========================================
# 2) Imports
# ========================================
import tensorflow as tf
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, classification_report
import numpy as np

# ========================================
# 3) Data Generators
# ========================================
IMG_SIZE = 224
BATCH_SIZE = 32
DATA_DIR = "/kaggle/input/structural-defects-network-concrete-crack-images/Decks"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# ========================================
# 4) Build EfficientNetB0 Model
# ========================================
base_model = efn.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False  # freeze feature extractor

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ========================================
# 5) Train Model
# ========================================
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)

# ========================================
# 6) Evaluate F1 Score
# ========================================
# Get predictions
val_gen.reset()
preds = (model.predict(val_gen) > 0.5).astype(int)
true_labels = val_gen.classes

# Classification report
print(classification_report(true_labels, preds, digits=4))
print("F1 Score:", f1_score(true_labels, preds))


Found 10896 images belonging to 2 classes.
Found 2724 images belonging to 2 classes.


I0000 00:00:1754773755.666273      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


16804768/16804768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


I0000 00:00:1754773773.687620     134 service.cc:148] XLA service 0x794f8814d1a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754773773.688391     134 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754773775.392833     134 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/341 ━━━━━━━━━━━━━━━━━━━━ 2:22:01 25s/step - accuracy: 0.9062 - loss: 0.5638

I0000 00:00:1754773784.206635     134 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


341/341 ━━━━━━━━━━━━━━━━━━━━ 95s 204ms/step - accuracy: 0.8786 - loss: 0.3376 - val_accuracy: 0.9174 - val_loss: 0.2498
Epoch 2/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 24s 72ms/step - accuracy: 0.9070 - loss: 0.2721 - val_accuracy: 0.9159 - val_loss: 0.2525
Epoch 3/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 27s 78ms/step - accuracy: 0.9164 - loss: 0.2557 - val_accuracy: 0.9123 - val_loss: 0.2767
Epoch 4/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 27s 80ms/step - accuracy: 0.9132 - loss: 0.2521 - val_accuracy: 0.9236 - val_loss: 0.2379
Epoch 5/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 26s 75ms/step - accuracy: 0.9131 - loss: 0.2531 - val_accuracy: 0.9214 - val_loss: 0.2414
86/86 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step
              precision    recall  f1-score   support

           0     0.8577    0.5654    0.6815       405
           1     0.9284    0.9836    0.9552      2319

    accuracy                         0.9214      2724
   macro avg     0.8930    0.7745    0.8184      2724
weighted avg     0.9179    0.9214    0.9145      2

In [13]:
# ========================================
# 1) Install dependencies
# ========================================
!pip install -q tensorflow scikit-learn segmentation-models

# ========================================
# 2) Imports
# ========================================
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import segmentation_models as sm

# Set image size
IMG_HEIGHT, IMG_WIDTH = 256, 256
DATASET_PATH = "/kaggle/input/concrete-crack-segmentation-dataset/concreteCrackSegmentationDataset"  # change this to your dataset path

# ========================================
# 3) Load Data
# ========================================
import os
import numpy as np
from tensorflow.keras.utils import img_to_array, load_img

IMG_HEIGHT, IMG_WIDTH = 256, 256

image_dir = "/kaggle/input/concrete-crack-segmentation-dataset/concreteCrackSegmentationDataset/rgb"
mask_dir  = "/kaggle/input/concrete-crack-segmentation-dataset/concreteCrackSegmentationDataset/BW"

# Keep only jpg files
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(".jpg")]
mask_files  = [f for f in os.listdir(mask_dir) if f.lower().endswith(".jpg")]

# Convert to name without extension for comparison
image_ids = set(os.path.splitext(f)[0] for f in image_files)
mask_ids  = set(os.path.splitext(f)[0] for f in mask_files)

# Intersection — only the IDs present in BOTH
common_ids = sorted(image_ids & mask_ids)
print(f"✅ Found {len(common_ids)} matching image-mask pairs.")

images, masks = [], []

for file_id in common_ids:
    img_path = os.path.join(image_dir, file_id + ".jpg")
    mask_path = os.path.join(mask_dir, file_id + ".jpg")
    
    # Safety check
    if not os.path.exists(img_path) or not os.path.exists(mask_path):
        continue

    img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    img = img_to_array(img) / 255.0
    images.append(img)

    mask = load_img(mask_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")
    mask = img_to_array(mask) / 255.0
    masks.append(mask)

images = np.array(images)
masks = np.array(masks)

print("Images shape:", images.shape)
print("Masks shape:", masks.shape)



# ========================================
# 4) Train-test split
# ========================================
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# ========================================
# 5) Build U-Net
# ========================================
sm.set_framework('tf.keras')
sm.framework()

# Use U-Net with a pretrained backbone
BACKBONE = 'efficientnetb0'
preprocess_input = sm.get_preprocessing(BACKBONE)

X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=1, activation='sigmoid')

# Dice loss + binary crossentropy
dice_loss = sm.losses.DiceLoss()
bce_loss  = sm.losses.BinaryCELoss() 
total_loss = dice_loss + bce_loss

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(optimizer='adam', loss=total_loss, metrics=metrics)

# ========================================
# 6) Train
# ========================================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=8
)

# ========================================
# 7) Evaluate
# ========================================
scores = model.evaluate(X_val, y_val, verbose=0)
print(f"Mean IoU: {scores[1]:.4f}")
print(f"Dice Score: {scores[2]:.4f}")

✅ Found 446 matching image-mask pairs.
Images shape: (257, 256, 256, 3)
Masks shape: (257, 256, 256, 1)
Epoch 1/5


E0000 00:00:1754775569.727740     137 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1754775569.912392     137 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1754775570.322753     137 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1754775570.527636     137 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


25/26 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - f1-score: 0.3606 - iou_score: 0.2323 - loss: 1.1952

E0000 00:00:1754775615.431009     135 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1754775615.615639     135 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1754775616.018603     135 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1754775616.223432     135 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


26/26 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - f1-score: 0.3704 - iou_score: 0.2402 - loss: 1.1844 - val_f1-score: 0.0307 - val_iou_score: 0.0157 - val_loss: 3.9793
Epoch 2/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - f1-score: 0.6713 - iou_score: 0.5064 - loss: 0.7025 - val_f1-score: 0.0405 - val_iou_score: 0.0208 - val_loss: 1.9932
Epoch 3/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - f1-score: 0.7395 - iou_score: 0.5882 - loss: 0.4126 - val_f1-score: 0.0362 - val_iou_score: 0.0186 - val_loss: 3.2824
Epoch 4/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - f1-score: 0.7843 - iou_score: 0.6459 - loss: 0.2942 - val_f1-score: 7.2057e-04 - val_iou_score: 3.6046e-04 - val_loss: 1.0666
Epoch 5/5
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - f1-score: 0.7805 - iou_score: 0.6422 - loss: 0.2790 - val_f1-score: 0.0196 - val_iou_score: 0.0099 - val_loss: 1.1643
Mean IoU: 0.0097
Dice Score: 0.0192


In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json

In [2]:
import os
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
os.system("mv kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


256